In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
import warnings
import time
warnings.filterwarnings('ignore')
print("# Housing Price Growth Prediction Model")
print("## Approach #4: Improved Logistic Regression Model")
df = pd.read_csv("everything/everything.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['RegionName'] = df['RegionName'].astype(str)
df = df[(df['date'].dt.year >= 2007) & (df['date'].dt.year <= 2023)].copy()
def safe_temporal_operations(group):
    for lag in [1, 3, 6]:
        group[f'ZHVI_lag{lag}'] = group['ZHVI'].shift(lag)
    group['pct_change_1m'] = group['ZHVI'].pct_change(periods=1)
    group['pct_change_3m'] = group['ZHVI'].pct_change(periods=3)
    group['pct_change_6m'] = group['ZHVI'].pct_change(periods=6)
    group['ZHVI_3m_avg'] = group['ZHVI'].shift(1).rolling(window=3).mean()
    group['ZHVI_6m_avg'] = group['ZHVI'].shift(1).rolling(window=6).mean()
    group['ZHVI_3m_vol'] = group['ZHVI'].shift(1).rolling(window=3).std()
    group['ZHVI_6m_vol'] = group['ZHVI'].shift(1).rolling(window=6).std()
    group['ROC_3m'] = group['ZHVI'].pct_change(3) / 3  # Average monthly change over 3 months
    group['ROC_6m'] = group['ZHVI'].pct_change(6) / 6  # Average monthly change over 6 months
    group['target_pct_change'] = group['ZHVI'].pct_change(periods=1).shift(-1)
    group['month'] = group['date'].dt.month
    group['quarter'] = group['date'].dt.quarter
    
    return group
    
  
print("Creating temporal features...")
df = df.groupby('RegionName', group_keys=False).apply(safe_temporal_operations)
df = df.dropna(subset=['target_pct_change'])
df['target_binary'] = (df['target_pct_change'] > 0).astype(int)
train_end = pd.to_datetime('2022-11-01')
val_end   = pd.to_datetime('2023-05-01')
test_end  = pd.to_datetime('2023-11-01')

df_train = df[df['date'] < train_end].copy()
df_val   = df[(df['date'] >= train_end) & (df['date'] < val_end)].copy()
df_test  = df[(df['date'] >= val_end) & (df['date'] < test_end)].copy()

print(f"Train set: {df_train.shape[0]} rows, from {df_train['date'].min().date()} to {df_train['date'].max().date()}")
print(f"Validation set: {df_val.shape[0]} rows, from {df_val['date'].min().date()} to {df_val['date'].max().date()}")
print(f"Test set: {df_test.shape[0]} rows, from {df_test['date'].min().date()} to {df_test['date'].max().date()}")
core_features = [
    'ZHVI_lag1', 'ZHVI_lag3', 'ZHVI_lag6',
    'pct_change_1m', 'pct_change_3m', 'pct_change_6m',
    'ZHVI_3m_avg', 'ZHVI_6m_avg',
    'ZHVI_3m_vol', 'ZHVI_6m_vol',
    'ROC_3m', 'ROC_6m'
]
macro_features = [
    'Mortgage_rate', 'CPI', 'CPI_Rent', 'Unemp_rate', 'NASDAQ', 
    'disposable_income', 'Personal_consumption_expenditure', 'personal_savings'
]

location_features = ['SizeRank', 'Percent___of_State_Area_s_Population', 'Percent___of_Labor_Force_Unemployed_in_State_Area']

time_features = ['Year', 'month', 'quarter']
all_features = core_features + macro_features + location_features + time_features
available_features = [col for col in all_features if col in df_train.columns]
print(f"\nAvailable features: {len(available_features)} out of {len(all_features)}")
df_train['Year'] = df_train['date'].dt.year
df_val['Year'] = df_val['date'].dt.year
df_test['Year'] = df_test['date'].dt.year
if 'month' not in df_train.columns:
    df_train['month'] = df_train['date'].dt.month
    df_val['month'] = df_val['date'].dt.month
    df_test['month'] = df_test['date'].dt.month

if 'quarter' not in df_train.columns:
    df_train['quarter'] = df_train['date'].dt.quarter
    df_val['quarter'] = df_val['date'].dt.quarter
    df_test['quarter'] = df_test['date'].dt.quarter

def preprocess_data(df, features, train_mode=False):
    processed_df = df.copy()
    cat_features = ['RegionName']
    for cat in cat_features:
        if cat in processed_df.columns:
            if train_mode:
                dummies = pd.get_dummies(processed_df[cat], prefix=cat, drop_first=True)
                global CATEGORIES
                CATEGORIES = {cat: list(dummies.columns)}
            else:
                dummies = pd.get_dummies(processed_df[cat], prefix=cat)
                for col in CATEGORIES[cat]:
                    if col not in dummies.columns:
                        dummies[col] = 0
                dummies = dummies[CATEGORIES[cat]]
            
            processed_df = pd.concat([processed_df, dummies], axis=1)
    if train_mode:
        season_dummies = pd.get_dummies(processed_df['month'].apply(lambda x: (x%12)//3 + 1), prefix='season', drop_first=True)
        global SEASON_COLUMNS
        SEASON_COLUMNS = list(season_dummies.columns)
    else:
        season_dummies = pd.get_dummies(processed_df['month'].apply(lambda x: (x%12)//3 + 1), prefix='season')
        for col in SEASON_COLUMNS:
            if col not in season_dummies.columns:
                season_dummies[col] = 0
        season_dummies = season_dummies[SEASON_COLUMNS]
    
    processed_df = pd.concat([processed_df, season_dummies], axis=1)
    dummy_features = []
    if 'RegionName' in cat_features:
        dummy_features.extend(CATEGORIES['RegionName'])
    dummy_features.extend(SEASON_COLUMNS)
    features_to_use = [f for f in features if f in processed_df.columns] + dummy_features
    X = processed_df[features_to_use]
    y = processed_df['target_binary'] if 'target_binary' in processed_df.columns else None
    
    return X, y

print("\nPerforming feature selection...")
X_train_raw, y_train = preprocess_data(df_train, available_features, train_mode=True)
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train_raw)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selector.fit(X_train_scaled, y_train)
importances = rf_selector.feature_importances_
indices = np.argsort(importances)[::-1]
print("\nTop 15 most important features:")
for i in range(min(15, len(indices))):
    print(f"{X_train_raw.columns[indices[i]]}: {importances[indices[i]]:.4f}")

selector = SelectFromModel(rf_selector, prefit=True, threshold='mean')
X_train_selected = selector.transform(X_train_scaled)
selected_indices = selector.get_support()
selected_features = X_train_raw.columns[selected_indices].tolist()

print(f"\nSelected {len(selected_features)} features from {X_train_raw.shape[1]} total features")
print("Selected features:", selected_features)
X_val_raw, y_val = preprocess_data(df_val, available_features, train_mode=False)
X_val_imputed = imputer.transform(X_val_raw)
X_val_scaled = scaler.transform(X_val_imputed)
X_val_selected = selector.transform(X_val_scaled)
X_test_raw, y_test = preprocess_data(df_test, available_features, train_mode=False)
X_test_imputed = imputer.transform(X_test_raw)
X_test_scaled = scaler.transform(X_test_imputed)
X_test_selected = selector.transform(X_test_scaled)

print("\nTraining logistic regression with cross-validation...")
C_values = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
best_C = None
best_score = -np.inf
for C in C_values:
    model = LogisticRegression(C=C, max_iter=1000, solver='liblinear', random_state=42)
    cv_scores = cross_val_score(model, X_train_selected, y_train, cv=5, scoring='roc_auc')
    mean_score = np.mean(cv_scores)
    print(f"C={C}: Mean ROC-AUC: {mean_score:.4f}")
    
    if mean_score > best_score:
        best_score = mean_score
        best_C = C

print(f"Best C value: {best_C} with ROC-AUC: {best_score:.4f}")
final_model = LogisticRegression(C=best_C, max_iter=1000, solver='liblinear', random_state=42)
final_model.fit(X_train_selected, y_train)
coefficients = pd.DataFrame({
    'Feature': [X_train_raw.columns[i] for i in range(len(X_train_raw.columns)) if selected_indices[i]],
    'Coefficient': final_model.coef_[0]
})
coefficients = coefficients.sort_values('Coefficient', ascending=False)

print("\nTop positive coefficients:")
print(coefficients.head(10))
print("\nTop negative coefficients:")
print(coefficients.tail(10))
def evaluate_model(model, X, y, set_name):
    y_prob = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)
    print(f"\n{set_name} Performance:")
    print(classification_report(y, y_pred))
    roc_auc = roc_auc_score(y, y_prob)
    print(f"ROC-AUC: {roc_auc:.4f}")
    fpr, tpr, _ = roc_curve(y, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'{set_name} ROC Curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {set_name}')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.savefig(f'roc_curve_{set_name.lower()}.png')
    plt.close()
    
    return roc_auc, y_prob
train_auc, train_preds = evaluate_model(final_model, X_train_selected, y_train, "Training")
val_auc, val_preds = evaluate_model(final_model, X_val_selected, y_val, "Validation")
test_auc, test_preds = evaluate_model(final_model, X_test_selected, y_test, "Test")
plt.figure(figsize=(12, 8))
coefficients.sort_values('Coefficient').plot(x='Feature', y='Coefficient', kind='barh')
plt.title('Feature Coefficients')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('feature_coefficients.png')
plt.close()
plt.figure(figsize=(10, 6))
plt.hist(train_preds, alpha=0.5, bins=20, label='Train')
plt.hist(val_preds, alpha=0.5, bins=20, label='Validation')
plt.hist(test_preds, alpha=0.5, bins=20, label='Test')
plt.title('Prediction Probability Distribution')
plt.xlabel('Predicted Probability')
plt.ylabel('Count')
plt.legend()
plt.savefig('prediction_distribution.png')
plt.close()
print("\nPrediction Summary:")
print(f"Train: Mean={np.mean(train_preds):.4f}, Min={np.min(train_preds):.4f}, Max={np.max(train_preds):.4f}")
print(f"Validation: Mean={np.mean(val_preds):.4f}, Min={np.min(val_preds):.4f}, Max={np.max(val_preds):.4f}")
print(f"Test: Mean={np.mean(test_preds):.4f}, Min={np.min(test_preds):.4f}, Max={np.max(test_preds):.4f}")
df_test_copy = df_test.copy()
df_test_copy['predicted_prob'] = test_preds
df_test_copy['predicted_class'] = (test_preds >= 0.5).astype(int)
df_test_copy['correct'] = (df_test_copy['predicted_class'] == df_test_copy['target_binary']).astype(int)
region_accuracy = df_test_copy.groupby('RegionName')['correct'].mean().sort_values(ascending=False)
print("\nTop 10 regions by prediction accuracy:")
print(region_accuracy.head(10))
print("\nBottom 10 regions by prediction accuracy:")
print(region_accuracy.tail(10))
region_growth_prob = df_test_copy.groupby('RegionName')['predicted_prob'].mean().sort_values(ascending=False)
top_regions = region_growth_prob.head(10)
plt.figure(figsize=(12, 8))
top_regions.plot(kind='bar')
plt.title('Top 10 Regions by Predicted Growth Probability')
plt.ylabel('Average Predicted Probability')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('top_regions_by_growth.png')
plt.close()

print("\nTop 10 regions by predicted growth probability:")
print(top_regions)
results_df = pd.DataFrame({
    'RegionName': df_test['RegionName'],
    'date': df_test['date'],
    'actual_binary': df_test['target_binary'],
    'predicted_prob': test_preds,
    'predicted_binary': (test_preds >= 0.5).astype(int)
})
results_df.to_csv('housing_price_growth_predictions.csv', index=False)

print("\nResults saved to housing_price_growth_predictions.csv")
print("Model and visualizations have been saved.")
print("\nModel Training Complete!")


# Housing Price Growth Prediction Model
## Approach #4: Improved Logistic Regression Model


FileNotFoundError: [Errno 2] No such file or directory: 'everything/everything.csv'